In [ ]:
import math
import pandas as pd
from datetime import datetime

In [ ]:
user_ids = [1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 37, 38, 39, 40, 44, 45, 46, 47, 48]
len(user_ids)

35

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#add and preprocess data

In [ ]:
db = '/content/drive/MyDrive/reliance/data_analysis_files+code/'

# add processed data at 50%/65%/75%/85%
percent_buckets = 0.9
duration = pd.read_csv(db+'bucket_gaze_duration_divided_by_fixation_total_{}.csv'.format(percent_buckets), sep=',')
shift = pd.read_csv(db+'bucket_gaze_shift_{}.csv'.format(percent_buckets), sep=',')
shift_origin = shift.copy()

# add processed data at 100%
total_features = pd.read_excel(db+'features.xlsx')
total_features = total_features.sort_values(by = ['User id', 'round'])
total_features = total_features.reset_index(drop=True)

In [ ]:
#extract features from duration data
percent_duration_ai = duration.loc[(duration['location'] == 'AI')& (duration['bucket'] == 0)][['percent duration', 'User id', 'round']]
percent_duration_task = duration.loc[(duration['location'] == 'Task')& (duration['bucket'] == 0)][['percent duration', 'User id', 'round']]
percent_duration_user = duration.loc[(duration['location'] == 'User')& (duration['bucket'] == 0)][['percent duration', 'User id', 'round']]

#extract features from shift data
percent_shift_ai2task = shift.loc[(shift['shift'] == 'AI2Task') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_ai2user = shift.loc[(shift['shift'] == 'AI2User') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_task2user = shift.loc[(shift['shift'] == 'Task2User') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_task2ai = shift.loc[(shift['shift'] == 'Task2AI') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']] 
percent_shift_user2ai = shift.loc[(shift['shift'] == 'User2AI') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_user2task = shift.loc[(shift['shift'] == 'User2Task') & (shift['bucket'] == 0)][['percent_shift', 'User id', 'round']]

shift.loc[shift['shift'] == 'Task2User', 'shift'] = '2User'
shift.loc[shift['shift'] == 'AI2User', 'shift'] = '2User'
shift.loc[shift['shift'] == 'User2Task', 'shift'] = '2Task'
shift.loc[shift['shift'] == 'AI2Task', 'shift'] = '2Task'
shift.loc[shift['shift'] == 'User2AI', 'shift'] = '2AI'
shift.loc[shift['shift'] == 'Task2AI', 'shift'] = '2AI'

shift_less = shift.groupby(['User id', 'round', 'bucket', 'shift'], as_index=False)[['percent_shift', 'shift_per_sec']].sum()
shift_by = shift[['User id', 'round', 'orientation', 'Acc']].groupby(['User id', 'round'], as_index=False).first()
shift_less_merged = shift_less.merge(shift_by[['User id', 'round', 'orientation', 'Acc']], left_on=["User id", 'round'], right_on=["User id", 'round'])

percent_shift_2task = shift_less_merged.loc[(shift_less_merged['shift'] == '2Task') & (shift_less_merged['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_2ai = shift_less_merged.loc[(shift_less_merged['shift'] == '2AI') & (shift_less_merged['bucket'] == 0)][['percent_shift', 'User id', 'round']]
percent_shift_2user = shift_less_merged.loc[(shift_less_merged['shift'] == '2User') & (shift_less_merged['bucket'] == 0)][['percent_shift', 'User id', 'round']]

In [ ]:
shift

,User id,round,bucket,shift,count,shift_total_bucket,after_ai_appear_time,percent_shift,shift_per_sec,orientation,Acc
0,1,1,0,2Task,3,17,17.0,0.176471,0.196078,top,1
1,1,1,0,2User,1,17,17.0,0.058824,0.065359,top,1
2,1,1,0,2AI,1,17,17.0,0.058824,0.065359,top,1
3,1,1,0,2User,5,17,17.0,0.294118,0.326797,top,1
4,1,1,0,2AI,3,17,17.0,0.176471,0.196078,top,1
...,...,...,...,...,...,...,...,...,...,...,...
4159,48,8,1,2AI,0,0,NaN,0.000000,0.000000,bottom,1
4160,48,9,0,2Task,0,0,NaN,0.000000,0.000000,top,1
4161,48,9,1,2Task,0,0,NaN,0.000000,0.000000,top,1
4162,48,9,1,2AI,0,0,NaN,0.000000,0.000000,top,1


#predicting using 100% time data

##linear regression to predict final user ai agreement

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

test_features = total_features[["User id", "total_duration", #"percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                #"percent_duration_task", 
                                "percent_duration_user", 
                                #"first_ai_start_time", "first_ai_duration", 
                                #"percent_shift_ai2task", "percent_shift_ai2user", 
                                #"percent_shift_task2user", "percent_shift_task2ai", 
                                #"percent_shift_user2ai", "percent_shift_user2task", 
                                #"percent_shift_2task", 
                                "percent_shift_2ai", 
                                #"percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", #"start_user_truth_acc", 
                                "start_user_ai_acc", 
                                #"Orientation", 
                                ]]
test_features.loc[total_features['Gender'] == 'male', 'Gender'] = 0
test_features.loc[total_features['Gender'] == 'female', 'Gender'] = 1
test_features.loc[total_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_features.loc[total_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_features
y = total_features[["final_user_ai_acc"]]

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y)
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)
'''
X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''
'''
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
'''
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_train)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_train)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MSE:",mse)

MSE: 0.024182872748399066


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


##linear regression to predict percent of gaze duration in ai

In [ ]:
test_features = total_features[["User id", #"total_duration", "percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                "percent_duration_task", 
                                #"percent_duration_user", 
                                #"first_ai_start_time", 
                                #"first_ai_duration", 
                                "percent_shift_ai2task", "percent_shift_ai2user", 
                                "percent_shift_task2user", "percent_shift_task2ai", 
                                "percent_shift_user2ai", 
                                #"percent_shift_user2task", 
                                #"percent_shift_2task", "percent_shift_2ai", "percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", #"start_user_truth_acc", 
                                #"start_user_ai_acc", "Orientation", 
                                #"acc_improvement", "similarity_improvement"
                                ]]
test_features.loc[total_features['Gender'] == 'male', 'Gender'] = 0
test_features.loc[total_features['Gender'] == 'female', 'Gender'] = 1
test_features.loc[total_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_features.loc[total_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_features
y = total_features[["percent_duration_ai"]]

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y)
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

'''
X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)


X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''
'''
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)

mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

'''
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_train)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_train)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MSE:",mse)

MSE: 0.0013700163215333134


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


##ordinal regression to predict survey reliance

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

test_features = total_features[["User id", "total_duration", 
                                #"percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                #"percent_duration_task", 
                                "percent_duration_user", 
                                #"first_ai_start_time", "first_ai_duration", 
                                "percent_shift_ai2task", #"percent_shift_ai2user", 
                                "percent_shift_task2user", #"percent_shift_task2ai", 
                                "percent_shift_user2ai", #"percent_shift_user2task", 
                                #"percent_shift_2task", "percent_shift_2ai", 
                                "percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", 
                                "start_user_ai_acc", 
                                #"start_user_truth_acc", 
                                "Orientation", 
                                ]]
test_features.loc[total_features['Gender'] == 'male', 'Gender'] = 0
test_features.loc[total_features['Gender'] == 'female', 'Gender'] = 1
test_features.loc[total_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_features.loc[total_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_features
y = total_features[["survey_reliance"]]

X_scaler = StandardScaler().fit(X)
X_scaled = X_scaler.fit_transform(X)

y.loc[total_features["survey_reliance"] < 5, "survey_reliance"] = 0
y.loc[total_features["survey_reliance"] >= 5, "survey_reliance"] = 1
y_scaled = y

'''
X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]

X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

log_regression = LogisticRegression()
log_regression.fit(X_train, y_train)

RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RF.fit(X_train, y_train)

y_pred = log_regression.predict_proba(X_train)
y_pred = y_pred[:,1]

y_pred = (y_pred > .5).astype(int)

accuracy = accuracy_score(y_train, y_pred)
print(accuracy)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

0.6637554585152838


#predicting using 0-90 time data

**add new features depending on time**

In [ ]:
new_features = percent_duration_ai.merge(total_features, how='inner', on=['User id', 'round'])
new_features['percent_duration_ai']=new_features['percent duration']
new_features = new_features.drop(columns=['percent duration'])

new_features = percent_duration_task.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_duration_task']=new_features['percent duration']
new_features = new_features.drop(columns=['percent duration'])

new_features = percent_duration_user.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_duration_user']=new_features['percent duration']
new_features = new_features.drop(columns=['percent duration'])

In [ ]:
new_features = percent_shift_ai2task.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_ai2task']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_ai2user.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_ai2user']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_user2task.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_user2task']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_user2ai.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_user2ai']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_task2ai.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_task2ai']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_task2user.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_task2user']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

In [ ]:
new_features = percent_shift_2task.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_2task']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_2user.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_2user']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features = percent_shift_2ai.merge(new_features, how='inner', on=['User id', 'round'])
new_features['percent_shift_2ai']=new_features['percent_shift']
new_features = new_features.drop(columns=['percent_shift'])

new_features['total_duration']=new_features['total_duration'] * percent_buckets

In [ ]:
new_features

,User id,round,Unnamed: 0,total_duration,percent_duration_ai,total_duration_on_ai,Id,Round,survey_agreement,survey_reliance,...,count_2ai,count_2user,final_user_ai_acc,final_user_truth_acc,ai_truth_acc,start_user_ai_acc,start_user_truth_acc,Orientation,acc_improvement,similarity_improvement
0,1,1,0,37.8,0.143479,2.73363,1,1,6,5,...,4,6,1.0000,1.0000,1.0000,0.833333,0.833333,top,0.166667,0.166667
1,1,2,2,300.6,0.068401,6.38815,2,2,1,1,...,12,82,0.3750,0.3750,0.9375,0.750000,0.750000,bottom,-0.375000,-0.375000
2,1,3,3,42.3,0.283739,5.09594,3,3,6,6,...,8,7,1.0000,0.9375,0.9375,1.000000,0.916667,top,0.020833,0.000000
3,1,4,4,56.7,0.156498,2.92737,4,4,1,1,...,3,17,0.3750,0.3750,1.0000,0.666667,0.666667,bottom,-0.291667,-0.291667
4,1,5,5,32.4,0.225435,2.79822,5,5,7,7,...,7,10,1.0000,1.0000,1.0000,0.916667,0.916667,top,0.083333,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,48,6,338,42.3,0.135470,3.41316,456,6,6,6,...,4,15,1.0000,1.0000,1.0000,0.750000,0.750000,bottom,0.250000,0.250000
338,48,7,339,43.2,0.097487,1.22974,457,7,2,2,...,3,17,0.9375,1.0000,0.9375,0.833333,0.916667,top,0.083333,0.104167
339,48,8,340,35.1,0.133005,2.94226,458,8,2,2,...,5,10,0.9375,1.0000,0.9375,0.833333,0.916667,bottom,0.083333,0.104167
340,48,9,341,31.5,0.059710,1.39295,459,9,6,2,...,3,14,1.0000,1.0000,1.0000,0.916667,0.916667,top,0.083333,0.083333


In [ ]:
total_features

,Unnamed: 0,User id,round,total_duration,percent_duration_ai,total_duration_on_ai,Id,Round,survey_agreement,survey_reliance,...,count_2ai,count_2user,final_user_ai_acc,final_user_truth_acc,ai_truth_acc,start_user_ai_acc,start_user_truth_acc,Orientation,acc_improvement,similarity_improvement
0,0,1,1,42,0.185714,2.73363,1,1,6,5,...,4,6,1.0000,1.0000,1.0000,0.833333,0.833333,top,0.166667,0.166667
1,2,1,2,334,0.024003,6.38815,2,2,1,1,...,12,82,0.3750,0.3750,0.9375,0.750000,0.750000,bottom,-0.375000,-0.375000
2,3,1,3,47,0.332952,5.09594,3,3,6,6,...,8,7,1.0000,0.9375,0.9375,1.000000,0.916667,top,0.020833,0.000000
3,4,1,4,63,0.076400,2.92737,4,4,1,1,...,3,17,0.3750,0.3750,1.0000,0.666667,0.666667,bottom,-0.291667,-0.291667
4,5,1,5,36,0.201638,2.79822,5,5,7,7,...,7,10,1.0000,1.0000,1.0000,0.916667,0.916667,top,0.083333,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,338,48,6,47,0.130746,3.41316,456,6,6,6,...,4,15,1.0000,1.0000,1.0000,0.750000,0.750000,bottom,0.250000,0.250000
338,339,48,7,48,0.040987,1.22974,457,7,2,2,...,3,17,0.9375,1.0000,0.9375,0.833333,0.916667,top,0.083333,0.104167
339,340,48,8,39,0.153503,2.94226,458,8,2,2,...,5,10,0.9375,1.0000,0.9375,0.833333,0.916667,bottom,0.083333,0.104167
340,341,48,9,35,0.074685,1.39295,459,9,6,2,...,3,14,1.0000,1.0000,1.0000,0.916667,0.916667,top,0.083333,0.083333


##linear regression to predict final user ai agreement

In [ ]:
test_new_features = new_features[["User id","total_duration", #"percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                #"percent_duration_task", 
                                "percent_duration_user", 
                                #"first_ai_start_time", "first_ai_duration", 
                                #"percent_shift_ai2task", "percent_shift_ai2user", 
                                #"percent_shift_task2user", "percent_shift_task2ai", 
                                #"percent_shift_user2ai", "percent_shift_user2task", 
                                #"percent_shift_2task", 
                                "percent_shift_2ai", 
                                #"percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", #"start_user_truth_acc", 
                                "start_user_ai_acc", 
                                #"Orientation", 
                                ]]
'''
test_new_features = new_features[["User id",
                                "Age", "Gender", "Education", 
                                "Familiarity", "Trust", 

                                "Orientation", 
                                ]]
'''
test_new_features.loc[new_features['Gender'] == 'male', 'Gender'] = 0
test_new_features.loc[new_features['Gender'] == 'female', 'Gender'] = 1
test_new_features.loc[new_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_new_features.loc[new_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_new_features
y = total_features[["final_user_ai_acc"]]

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y)
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''

X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

'''
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
'''
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_train)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_train)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
'''
print("MSE:",mse)

MSE: 0.056786093215810934


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


##linear regression to predict percent gaze duration on ai

In [ ]:
test_new_features = new_features[["User id",#"total_duration", "percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                "percent_duration_task", 
                                #"percent_duration_user", 
                                #"first_ai_start_time", 
                                #"first_ai_duration", 
                                "percent_shift_ai2task", "percent_shift_ai2user", 
                                "percent_shift_task2user", "percent_shift_task2ai", 
                                "percent_shift_user2ai", #"percent_shift_user2task", 
                                #"percent_shift_2task", 
                                #"percent_shift_2ai", 
                                #"percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", #"start_user_truth_acc", 
                                #"start_user_ai_acc", 
                                #"Orientation", 
                                ]]
'''
test_new_features = new_features[["User id",
                                "Age", "Gender", "Education", 
                                "Familiarity", "Trust", 

                                "Orientation", 
                                ]]
'''
test_new_features.loc[new_features['Gender'] == 'male', 'Gender'] = 0
test_new_features.loc[new_features['Gender'] == 'female', 'Gender'] = 1
test_new_features.loc[new_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_new_features.loc[new_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_new_features
y = total_features[["percent_duration_ai"]]

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y)
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''

X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''

linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
'''
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred = linear_regression.predict(X_train)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_train)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
'''
print("MSE:",mse)

MSE: 0.00419674357629768


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


##ordinal regression to predict survey reliance


In [ ]:
test_new_features = new_features[["User id", "total_duration", 
                                #"percent_duration_ai", 
                                #"survey_reliance", "final_user_ai_acc","survey_agreement", "survey_trust", "survey_confidence", 
                                #"Age", "Gender", "Education", 
                                "Familiarity", #"Trust", 
                                #"percent_duration_task", 
                                "percent_duration_user", 
                                #"first_ai_start_time", "first_ai_duration", 
                                "percent_shift_ai2task", #"percent_shift_ai2user", 
                                "percent_shift_task2user", #"percent_shift_task2ai", 
                                "percent_shift_user2ai", #"percent_shift_user2task", 
                                #"percent_shift_2task", "percent_shift_2ai", 
                                "percent_shift_2user", 
                                #"final_user_truth_acc", "ai_truth_acc", 
                                "start_user_ai_acc", 
                                #"start_user_truth_acc", 
                                "Orientation", 
                                ]]
'''
test_new_features = new_features[["User id",
                                "Age", "Gender", "Education", 
                                "Familiarity", "Trust", 

                                "Orientation", 
                                ]]
'''
test_new_features.loc[total_features['Gender'] == 'male', 'Gender'] = 0
test_new_features.loc[total_features['Gender'] == 'female', 'Gender'] = 1
test_new_features.loc[total_features['Orientation'] == 'bottom', 'Orientation'] = 0
test_new_features.loc[total_features['Orientation'] == 'top', 'Orientation'] = 1

X = test_new_features
y = total_features[["survey_reliance"]]

X_scaler = StandardScaler().fit(X)
X_scaled = X_scaler.fit_transform(X)

y.loc[total_features["survey_reliance"] < 5, "survey_reliance"] = 0
y.loc[total_features["survey_reliance"] >= 5, "survey_reliance"] = 1
y_scaled = y

X_train = X_scaled[total_features['round'] < 8]
X_test = X_scaled[total_features['round'] >= 8]
y_train = y_scaled[total_features['round'] < 8]
y_test = y_scaled[total_features['round'] >= 8]
X_train, y_train = shuffle(X_train, y_train , random_state=33)
X_test, y_test = shuffle(X_test, y_test, random_state=33)
'''

X_train = X_scaled[total_features['User id'] < 36]
X_test = X_scaled[total_features['User id'] >= 36]
y_train = y_scaled[total_features['User id'] < 36]
y_test = y_scaled[total_features['User id'] >= 36]
X_train, y_train = shuffle(X_train, y_train , random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)
'''
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

log_regression = LogisticRegression()
log_regression.fit(X_train, y_train)

RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RF.fit(X_train, y_train)

y_pred = log_regression.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

0.5
